In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["HUGGINGFACEHUB_API_KEY"]=os.getenv("HUGGINGFACEHUB_API_TOKEN")
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# os.environ["PPLX_API_KEY"] = os.getenv("PPLX_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion - Scraping the website for content
from langchain_community.document_loaders import WebBaseLoader

c:\Users\Nitin Mishra\OneDrive\Desktop\AIML\Gen-AI\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader = WebBaseLoader("https://www.geeksforgeeks.org/machine-learning/ml-bias-variance-trade-off/")
loader


In [4]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://www.geeksforgeeks.org/machine-learning/ml-bias-variance-trade-off/', 'title': 'Bias-Variance Trade Off - Machine Learning - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBias-Variance Trade Off - Machine Learning - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\nCoursesDSA / PlacementsGATE PrepML & Data ScienceDevelopmentCloud / DevOpsProgramming LanguagesAll CoursesTutorialsPythonJavaDSAML & Data ScienceInterview CornerProgramming LanguagesWeb DevelopmentGATECS SubjectsDevO

Load Data(Documents)--> Divide into Chunks --> Create Embeddings --> Store in Vector DB --> Retrieve Relevant Docs using Similarity Search --> Use LLM to Answer Questions

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)

In [6]:
documents

[Document(metadata={'source': 'https://www.geeksforgeeks.org/machine-learning/ml-bias-variance-trade-off/', 'title': 'Bias-Variance Trade Off - Machine Learning - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content="Bias-Variance Trade Off - Machine Learning - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\nCoursesDSA / PlacementsGATE PrepML & Data ScienceDevelopmentCloud / DevOpsProgramming LanguagesAll CoursesTutorialsPythonJavaDSAML & Data ScienceInterview CornerProgramming LanguagesWeb DevelopmentGATECS SubjectsDevOpsSchool LearningSoftware and ToolsPracticePractice Coding ProblemsNation Skillup- Ending Soon!Problem

In [7]:
# Embeddings
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [8]:
# Creating Vector Store
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents,embeddings)

In [9]:
vectorstore

In [13]:
## Querying the vector store

query = "Bias and Variance Tradeoff"
result = vectorstore.similarity_search(query)
result

[Document(id='eb8e1638-a96d-4a3c-91c5-8ed3dfa06332', metadata={'source': 'https://www.geeksforgeeks.org/machine-learning/ml-bias-variance-trade-off/', 'title': 'Bias-Variance Trade Off - Machine Learning - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content='What is Bias?\nThe bias is known as the difference between the prediction of the values by the Machine Learning model and the correct value. Being high in biasing gives a large error in training as well as testing data. It recommended that an algorithm should always be low-biased to avoid the problem of underfitting. By high bias, the data predicted is in a straight line format, thus not fitting accurately in the data in the data set. Such fitting is known as the Und

In [14]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model = "gemma:2b")

RETRIEVAL CHAIN:
- A Retrieval Chain is a process used in GenAI where the system first searches for relevant information (context) from a database or documents, and then sends that information along with the user’s question to the LLM.
- This helps the model give more accurate and context-aware answers, especially when the model does not already know the information.

In [15]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

documents_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
documents_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| OllamaLLM(model='gemma:2b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [20]:
from langchain_core.documents import Document
documents_chain.invoke({
    "input":"Bias and Variance Tradeoff",
    "context":[Document(page_content=result[0].page_content + result[1].page_content + result[2].page_content + result[3].page_content)]
})

'Sure, here is the answer to the question:\n\nBias is the difference between the prediction of the values by the Machine Learning model and the correct value. Being high in biasing gives a large error in training as well as testing data.'

<h4>WE MAKE VECTORSTORE DATABASE AND CONVERT IT INTO RETRIEVER.</h4>

<h5>WHAT DOES RETRIEVER DO AFTER THAT?:-</h5>
We don’t want to manually choose or load documents every time someone asks a question.
Instead, we want the retriever (a search mechanism) to automatically find the most relevant documents based on the user’s query.

So when a question comes in:

1. Retriever searches all available documents

2. Picks only the most useful/relevant ones

3. Sends those selected documents (context) to the LLM

4. LLM uses that context to generate a better and accurate answer

<h5>Why is this important?</h5>

- Because the LLM doesn’t always know everything on its own, and it shouldn’t read all documents every time (that would be slow and costly).
Instead, it should only receive information that matters for the current question.

<h5>Simple Analogy</h5>

Imagine you have a large library. Instead of reading all the books whenever  someone asks a question:

- The retriever acts like a librarian

- You ask the librarian your question

- The librarian quickly finds only the few books or pages that matter

- Then you read those pages to answer the question correctly.

<h5>Final Simple Statement</h5>

- The retriever helps automatically find and send only the most relevant    documents to the LLM so that the answer is accurate and context-aware for  each question.

In [24]:
## Creating retriever

retriever = vectorstore.as_retriever()

from langchain_classic.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever, documents_chain)
retriever_chain


RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000116EFEEBB00>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            |

In [25]:
## Getting response from the retriever chain
response = retriever_chain.invoke({
    "input":"Bias and Variance Tradeoff"
})

response['answer']

'The bias is known as the difference between the prediction of the values by the Machine Learning model and the correct value.'